In [7]:
import os

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier


from skactiveml.classifier import SklearnClassifier
from skactiveml.stream import StreamRandomSampling, PeriodicSampling, FixedUncertainty
from skactiveml.utils import call_func, MISSING_LABEL

import warnings
warnings.filterwarnings('ignore')

In [4]:
init_train_ratio = 0.1
stream_ratio = 0.9

In [5]:
class WrappedXGBClassifier(XGBClassifier):
    def fit(self, X, y, **kwargs):
        return super().fit(X, y, **kwargs)
    def predict(self,X, **kwargs):
        return super().predict(X, **kwargs)
    def predict_proba(self, X, **kwargs):
        return super().predict_proba(X, **kwargs)
    def score(self, X, y, **kwargs):
        return super().score(X, y, **kwargs)

In [6]:
models = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter = 10000, random_state = 0),
        'params': {
            'C': [0.001, 0.01, 0.1, 0.5, 1, 5, 10, 100]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(random_state = 0),
        'params': {
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [100, 300, 500],
        'gamma': [0, 0.1, 0.2],
        'reg_lambda': [0, 1, 10],
        'reg_alpha': [0, 1, 10]
        }
    }
}

budgets = [0.05, 0.1, 0.2, 0.5, 1.0]

In [17]:
data_path = '../data/CC18'
datasets = sorted(os.listdir(data_path))

In [18]:
datasets

['Bioresponse.csv',
 'GesturePhaseSegmentationProcessed.csv',
 'Internet-Advertisements.csv',
 'JapaneseVowels.csv',
 'MiceProtein.csv',
 'PhishingWebsites.csv',
 'adult.csv',
 'analcatdata_authorship.csv',
 'analcatdata_dmft.csv',
 'balance-scale.csv',
 'bank-marketing.csv',
 'banknote-authentication.csv',
 'blood-transfusion-service-center.csv',
 'breast-w.csv',
 'car.csv',
 'churn.csv',
 'climate-model-simulation-crashes.csv',
 'cmc.csv',
 'cnae-9.csv',
 'connect-4.csv',
 'credit-approval.csv',
 'credit-g.csv',
 'cylinder-bands.csv',
 'diabetes.csv',
 'dna.csv',
 'dresses-sales.csv',
 'electricity.csv',
 'eucalyptus.csv',
 'first-order-theorem-proving.csv',
 'har.csv',
 'ilpd.csv',
 'isolet.csv',
 'jm1.csv',
 'jungle_chess_2pcs_raw_endgame_complete.csv',
 'kc1.csv',
 'kc2.csv',
 'kr-vs-kp.csv',
 'letter.csv',
 'madelon.csv',
 'mfeat-factors.csv',
 'mfeat-fourier.csv',
 'mfeat-karhunen.csv',
 'mfeat-morphological.csv',
 'mfeat-pixel.csv',
 'mfeat-zernike.csv',
 'nomao.csv',
 'numerai

In [ ]:
results_df = pd.DataFrame()

for dataset in datasets:
    print(f'Dataset: {dataset}')
    df = pd.read_csv(f'{data_path}/{dataset}')

    X = df.drop(columns=['y_target_class'])

    y = df['y_target_class']

    X_init_train = X.iloc[:round(X.shape[0]*init_train_ratio)]
    y_init_train = y.iloc[:round(y.shape[0]*init_train_ratio)]

    X_stream = X.iloc[round(X.shape[0]*init_train_ratio):round(X.shape[0]*stream_ratio)]
    y_stream = y.iloc[round(y.shape[0]*init_train_ratio):round(y.shape[0]*stream_ratio)]

    X_final_val = X.iloc[round(X.shape[0]*stream_ratio):]
    y_final_val = y.iloc[round(y.shape[0]*stream_ratio):]

    for model_name, model_config in models.items():
        print(f"\tModel: {model_name}")
        grid_search = GridSearchCV(model_config['model'], model_config['params'], cv=2, scoring='accuracy', n_jobs=-1, error_score = 'raise', refit=True)
        grid_search.fit(X_init_train, y_init_train)

        for budget in budgets:
            print(f'\t\tBudget: {budget}')
            query_strategies = {
                'FixedUncertainty': FixedUncertainty(budget = budget, random_state = 0),
                'StreamRandomSampling': StreamRandomSampling(budget = budget, random_state = 0),
                'PeriodicSampler': PeriodicSampling(budget = budget, random_state = 0),
            }

            for query_strategy_name, query_strategy in query_strategies.items():
                print(f'\t\t\tQuery strategy: {query_strategy_name}')

                X_final_train = X_init_train
                y_final_train = y_init_train

                if model_name == 'XGBoost':
                    pretrained_model = WrappedXGBClassifier(grid_search.best_estimator_)
                else:
                    pretrained_model = grid_search.best_estimator_

                clf = SklearnClassifier(pretrained_model)
                clf.fit(X_init_train, y_init_train)

                initial_accuracy = clf.score(X_final_val, y_final_val)

                if budget == 1.0:
                    X_final_train = pd.concat([X_final_train, X_stream], axis = 0, ignore_index=True)
                    y_final_train = pd.concat([y_final_train, y_stream], axis = 0, ignore_index=True)


                    final_trained_model = clf.fit(X_final_train, y_final_train)
                    final_accuracy = final_trained_model.score(X_final_val, y_final_val)

                else:
                    for (index, x_s), y_s in zip(X_stream.iterrows(),y_stream):
                        x_cand = x_s.to_frame().T
                        y_cand = y_s

                        queried_indices, utilities = call_func(
                            query_strategy.query,
                            candidates=x_cand,
                            clf=clf,
                            return_utilities=True,
                            fit_clf=False
                        )

                        budget_manager_param_dict = {"utilities": utilities}

                        call_func(query_strategy.update,
                                candidates = x_cand,
                                queried_indices = queried_indices,
                                budget_manager_param_dict=budget_manager_param_dict)

                        if len(queried_indices):
                            X_final_train = pd.concat([X_final_train, x_cand], axis = 0, ignore_index=True)
                            y_final_train = pd.concat([y_final_train, pd.Series(y_s)], axis = 0, ignore_index=True)


                    final_trained_model = clf.fit(X_final_train, y_final_train)
                    final_accuracy = final_trained_model.score(X_final_val, y_final_val)

                experiment_result = pd.DataFrame(data = {
                    'dataset': [dataset],
                    'model_name': [model_name],
                    'model_params': [str(grid_search.best_params_)],
                    'initial_score': [initial_accuracy],
                    'query_strategy': [query_strategy_name],
                    'budget': [budget],
                    'percentage_queried': [(X_final_train.shape[0] - X_init_train.shape[0])/(X_stream.shape[0])],
                    'final_accuracy': [final_accuracy]
                })

                results_df = pd.concat([results_df, experiment_result], axis = 0)
                results_df.to_csv('../data/results/CC18_results.csv')

Dataset: Bioresponse.csv
	Model: LogisticRegression
		Budget: 0.05
			Query strategy: FixedUncertainty


In [10]:
results_df

""
